<hr style="border-top: 3px double #8c8b8b;"/>
<br/>
<center><font size=7 color="green">データサイエンスII   第1回 実習</font><br/>
2022.4.12
</center>
<hr style="border-top: 3px double #8c8b8b;"/>


updated 2022.4.30 mode => median

## 最初に次を実行

In [ ]:
import numpy as np
import plotly.express as px

# 実数集合Dの表示
def plot1d(D,width=600,height=300):
    import plotly.express as px
    import numpy as np
    fig=px.scatter(x=D,y=np.zeros(len(D)))
    fig.update_layout(width=width,height=height)
    fig.show()
    

# 実数集合Dを、t未満とt以上で色を変えて表示    
def plot1d2(D,t,width=600,height=300):
    import plotly.express as px
    import numpy as np
    color=[(lambda x:"blue" if x<t else "red")(u) for u in D]
    fig=px.scatter(x=D,y=np.zeros(len(D)),color=color)
    fig.add_scatter(x=[t],y=[0])
    fig.update_layout(width=width,height=height)
    fig.show()
    
# １変数関数のグラフ表示, flist は表示する関数のリスト、labels はそれぞれのラベル, xrange はxの動く範囲
def showgraph(flist,labels,xrange=[0,1],K=50):
    import plotly.graph_objects as go
    import numpy as np
    fig = go.Figure()
    tmin,tmax=xrange
    ts=np.linspace(tmin,tmax,K)
    for i in range(len(flist)):
        fig.add_trace(go.Scatter(x=ts, y=[flist[i](t) for t in ts], name=labels[i]))
    if len(flist)==1:
        fig.update_layout(title=labels[0])
    fig.update_layout(width=600,height=600)

    fig.show() 

# ２変数関数fのグラフを表示
def showgraph2(f,xrange=[-1,1],yrange=[-1,1],K=50,opacity=0.5):
    import numpy as np
    import plotly.graph_objects as go
    xmin,xmax=xrange
    ymin,ymax=yrange
    a0=np.linspace(xmin,xmax,K)
    b0=np.linspace(ymin,ymax,K)
    xx,yy=np.meshgrid(a0,b0)
    F=go.Figure()
    F.add_surface(x=xx,y=yy,z=f(xx,yy),opacity=opacity)  
    F.update_layout(width=600,height=600)
    F.show()

def toukousen(f,xrange=[0,1],yrange=[0,1],T=20,start=0,end=1,size=1):
    import numpy as np
    import plotly.graph_objects as go
    xmin,xmax=xrange
    ymin,ymax=yrange
    a0=np.linspace(xmin,xmax,T)
    b0=np.linspace(ymin,ymax,T)
    x,y=np.meshgrid(a0,b0)
    F=go.Figure()
    F.add_contour(x=a0,y=b0,z=f(x,y),colorscale="oranges",
        contours=dict(
            start=start, #2 等高線の高さの最小値
            end=end,    #3 等高線の高さの最大値
            size=size, #4 描く間隔
            coloring='lines',
            showlabels=True
        ))
    F.update_yaxes(scaleanchor="x", scaleratio=1)
    F.update_layout(width=600,height=600)
    F.show()
def heikin(D):    
    return sum(D)/len(D)
def bunsan(D):
    if len(D)>0:
        m=heikin(D)
        return sum((D-m)**2)/len(D)
    else:
        return 0
def mode(D):
    D=sorted(D)
    k=len(D)//2
    if len(D)%2==1:
        return D[k]
    else:
        return (D[k-1]+D[k])/2.
def l1bunsan(D):
    if len(D)>0:
        m=mode(D)
        return sum(abs(D-m))
    else:
        return 0

<hr>

# 1 実習1. 実数データ集合を１実数で近似する問題

## 1.1 実数データを１実数で近似

実数のデータの集まり $D=\{d_0,d_1,\cdots,d_{N-1}\}$ 
を実数$a$で近似する問題を考えよう。これは$D$を代表する典型的な実数$a$を求めることにほかならない。
近似の度合はどのように定量化できるだろうか。

データが一つの場合、つまり$D=\{d\}$の場合は、$|d-a|$ が近似度を示しているが$|d-a|^2$ でも構わない。いずれにしても$0$となるのが最良の近似$d$を与えている。

しかし、$D$が複数あるときは、近似度のとり方は色々ありえる。
  $$gosa_2(x,D)=(x-d_0)^2+(x-d_1)^2+\cdots+(x-d_{N-1})^2$$
や
  $$gosa_1(x,D)=|x-d_0|+|x-d_1|+\cdots+|x-d_{N-1}|$$
が自然である。$gosa_i$を $L_i$誤差関数と呼ぶことにする。


$L_2$誤差関数を最小にする実数を求めてみよう。

## 1.2 L2誤差関数

$D=\{1,2,3,6,7,8,9\}$ の場合を調べる。これを numpy の配列とする。まず、numpy を読み込み、名称を `np`とする。

In [ ]:
import numpy as np

$D$は python のリスト `[1,2,3,6,7,8,9]`で表現できるが、 numpy の`ndarray` というデータ構造にしておくと便利なことが多い。このリストを`ndarray`にするには次のようにする。

In [ ]:
D=np.array([1,2,3,6,7,8,9])

In [ ]:
plot1d(D,height=300)

$a$と $D$の元$d$の差の二乗$(a-d)^2$は `(a-d)**2`と書く。これを成分とするリストは `[(a-d)**2 for d in D]` であった. このリストの成分の和が誤差関数となる.

In [ ]:
def gosa2(x,D):
    return sum([(x-d)**2 for d in D]) 

+ 注意. `[(x-d)**2 for d in D]`は簡潔に`(x-D)**2`とかけて、`gosa2`は `sum((x-D)**2)`でも定義できる。しかし、この表現を使うと、`x` がリストのときにエラーが生じるので、上のように定義する.

In [ ]:
gosa2(0,D),gosa2(5,D)

### 1.2.1 誤差関数のグラフ

関数`gosa2(x,D)`のグラフを描いてみる. 最初に$D$を固定して得られる１変数関数 $t\mapsto gosa2(t,D)$ を定義しておく. これはpython で次のように書く.

In [ ]:
gosa2D=lambda t:gosa2(t,D)

In [ ]:
showgraph([gosa2D],["L2誤差"],[0,10])

### 1.2.2 <font color="red">図の操作(重要)</font>

図はマウスで操作できる. メニューは、セルを選択したときに右上に現れる. 
+ ポインタに近いグラフの点の座標が表示される.
+ ズームイン：カーソルが+ ではないとき、メニュー２番目の虫眼鏡アイコンをクリック。
   + 左ボタンを押したままマウスを動かして範囲を指定. 
   + 上端からマウスを動かすと$x$の動く範囲を制限できる. 左端からだと$y$の範囲を制限できる.
+ 図の移動：カーソルが 
   + 水平方向に動かすには、下端にマウスを近づけアイコンが左右矢印になったら左ボタンを押したまま動かす。上下に動かすには左端で行う.  
   + 任意方向に動かすには、メニュー３番目の上下左右アイコンをクリックしたあと、左ボタンを押したまま動かす.
+ もとに戻すには、右上にポインタを置いてでてくるメニューの家アイコンをクリックする 

#### <font color="red">練習. 上の操作を自分でやってみよう。</font>

### 1.2.3 数式処理で最小化を行う

`gosa2(a)`は aの二次多項式なのでその最小値は微分法で簡単に求められる。
数式処理を行うライブラリ `sympy` を使って計算してみよう。

+ ライブラリ名を `sy`として読み込む

In [ ]:
import sympy as sy

+ `a`を変数として扱うには以下のようにする。

In [ ]:
from sympy.abc import a

+ 参考.
   + 複数のときは `from sympy.abc import a,b,theta`  
      theta は θと表示される
   + `a,b = sy.symbols("a b")` とも書く

+ 関数 gosa2 の式を `gs2`とする。

In [ ]:
gosa2D=gosa2(a,D)
gosa2D

 + 式を整理する。

In [ ]:
gosa2D.expand()

 + 二次式となったので、その最小値を求めるために微分する。

In [ ]:
gosa2D.diff(a)

 + これがゼロとなるaを求めるのに`solve` を用いる.

In [ ]:
sol=sy.solve(gosa2D.diff(a),a)
sol

+ 成分を取り出す

In [ ]:
a=sol[0]
a

+ 誤差を求める.

In [ ]:
gosa2(a,D)

### 1.2.4 命題. $a$は$D$の平均値. 誤差は分散.
+ 証明  $$\frac{\partial{gosa2(a)}}{\partial{a}}=
  \sum_{d\in D}2(a-d)=2(|D|a-\sum_{d\in D} d).$$
  よって、$\frac{\partial{gosa2(a)}}{\partial{a}}=0$ を$a$について解くと
  $$ a=\frac{\sum D}{|D|},$$
 となり、$a$が$D$の平均値であることがわかり、
$a$における$L_2$誤差
  $$\sum_{d\in D}(d-a)^2$$
は分散であることもわかる.
  



### 1.2.5 平均値と分散との比較

+ `a.n(10)` は、有理数`a`を10桁まで小数展開する.
+ `heikin(D)`は、実数データ`D`の平均値
+ 次で、gosa2Dを最小化する`a`が平均値であることを確認する.

In [ ]:
a.n(10),heikin(D),a.n(10)-heikin(D)

+ `bunsan(D)` は実数データ`D`の分散.
+ 次で、gosa2Dの最小値を`D`の個数で割ったものが`D`の分散であることがわかる.

In [ ]:
gosa2(a,D)/len(D),bunsan(D),gosa2(a,D)/len(D)-bunsan(D)

# 2 実習2 L1誤差関数

+ $L_1$誤差については、どの数が$D$を最もよく近似するであろうか。

In [ ]:
def gosa1(x,D):
    return sum([abs(x-d) for d in D])

##  2.1 誤差関数のグラフ

青のグラフが $L_1$誤差関数である.

In [ ]:
gosa1D=lambda t:gosa1(t,D)
gosa2D=lambda t:gosa2(t,D)
showgraph([gosa1D,gosa2D],["L1誤差","L2誤差"],[0,10],K=1000)

+ 赤が$L_2$誤差、青が$L_1$誤差
+ 右欄にある赤線をクリックすると青のグラフだけになる。
+ $L_1$誤差曲線は折れ線であり、$6$で最小となっている。
+ $L_2$近似での最良近似 $5.14285\dots$より大きい。

## 2.2 数値微分

+ $L_1$誤差関数 `gosa1`は微分可能でないので、`sympy`を使った§1.2,3 の方法は使えない。そこで数値微分を使って調べる。数値微分は、この講義の後半ではライブラリ`pytorch`を利用するが、当面は素朴に扱う。


+ $\varepsilon$を小さくしておけば
$$f'(x)=\lim_{h\rightarrow 0}\frac{f(x+h)-f(x)}{h}\approx \frac{f(x+\varepsilon)-f(x)}{\varepsilon}$$

+ 右辺を f,x,epsilon の関数として定義する.

In [ ]:
def ndiff(f,x,epsilon=0.01):
    return ((f(x+epsilon)-f(x))/epsilon)

+ $L_1$誤差関数`gosa1D`の数値導関数を`Dgosa1D`とする 

In [ ]:
Dgosa1D=lambda x:ndiff(gosa1D,x)

+ グラフを描く

In [ ]:
showgraph([gosa1D,Dgosa1D],["gosa1","gosa1の微分"],[0,10])

+ 青をクリックして微分のグラフだけ表示してみると、$x<6$で微分が負、$x>6$で正であることがわかり、$x=6$が最小値なことがわかる。これは$D$の中央値である.

+ 定義. $D=\{d_1\leq d_2\leq \cdots\leq d_N\}$の中央値は
    + $N=2k+1$のとき$d_k$
    + $N=2k$のとき、$(d_k+d_{k+1})/2$ 

# 3 実習3 実数データを２実数で近似する問題

## 3.1 誤差関数

実数の集合$D$ を最もよく近似する２実数$a,b$を求めてみよう。集合$A=\{a,b\}$への実数$x$の隔たり$d(A,x)$を、$x$に最も近い$A$の要素との隔たりとして定義することにする。２実数$x,a$の隔たりとして$|x-a|$をとると,
$$d_1(A,x)=\min_{a\in A}|x-a|$$
となる。平均値の場合と同様に、$x,a$の隔たりを$(x-a)^2$をとれば
$$d_2(A,x)=\min_{a\in A}(x-a)^2$$ となる。
$x$が$D$を動くとき、$x$と$A$の隔たりを足し合わせると$A$が$D$をどの程度近似しているかを表す指標となる。すなわち$L_2$誤差の一般化として
$$ gosa2_2(A,D)=\sum_{x\in D}\min_{a\in A}(x-a)^2$$
$L_1$誤差の一般化として
$$gosa2_1(A,D)=\sum_{x\in D}\min_{a\in A}|x-a|$$
と定めることができる。


## 3.2 L2誤差に関する近似

+ １点近似を行った$D=\{1,2,3,6,7,8,9\}$ を二点近似してみよう

In [ ]:
D=np.array([1,2,3,6,7,8,9])

### 3.2.1 誤差関数の定義

In [ ]:
def gosa22(a,b,D):
    return sum(min((a-d)**2,(b-d)**2) for d in D)

+ `D`は固定し`a,b`だけの関数としておく.

In [ ]:
gosa22D=lambda a,b: gosa22(a,b,D)

### 3.2.2 誤差関数のグラフ

 + 次によりpython 関数をndarray に適用できる関数にしておく. グラフを描くときに必要.

In [ ]:
gs22=np.frompyfunc(gosa22D,2,1)

In [ ]:
showgraph2(gs22,xrange=[0,10],yrange=[0,10])

#### <font color="red">練習. 図をマウスで拡大・縮小・回転してみよう</font>

### 3.2.3 関数の等高線表示

+ 次は２変数関数`gs22`を,$[0,10]\times [0,10]$の範囲で、等高線$gs22=0,0.5,1,1.5,...,10$を表示する.

In [ ]:
toukousen(gs22,[0, 10],[0,10],start=0,end=10,size=0.5,T=100)

+ $a,b$について対称なので、$a<b$の範囲を考える.
+ 左上の密なところをズームインすると$2,7.5$が最もよく近似していると推測される。
+ この場合のエラーは7となる。$2$は$1,2,3$の平均であり、$7.5$は$6,7,8,9$の平均となっている。

+ 誤差を計算すると

In [ ]:
gosa22(2,7.5,D)

+ これは、$1,2,3$の分散の3倍と、$6,7,8,9$の分散の4倍の和となっている.

In [ ]:
bunsan(np.array([1,2,3]))*3+bunsan(np.array([6,7,8,9]))*4

+ 今回も微分可能ではないので数式処理では解けない。

## 3.3 近似実数a,bによるDの二分割

+ a=2,b=7.5 とおく
+ $Da=\{x\in D | |x-a|<|x-b|\}$とおくと、これは$a$の方に近いデータの集合. その補集合を $Db$とする.

In [ ]:
a,b=2,7.5
Da=[u for u in D if abs(u-a)<abs(u-b)]
Db=[u for u in D if abs(u-a)>=abs(u-b)]
Da,Db

+ `Da` は $c=\frac{a+b}2=\frac{9.5}2 $より小さいもの、`Db`はそれより大きいものの集合、ともみれる
+ 次図は`Da`が青色、`Db`が赤、分割点が緑で示されている.

In [ ]:
plot1d2(D,9.5/2)

## 3.4 平均値・分散との関係

+ $2$は$Da$の平均値、$7.5$は  $Db$の平均値となっている. 

In [ ]:
print("a={:5.1f},Daの平均値={}\nb={:5.1f},Dbの平均値={}".format(a,heikin(Da),b,heikin(Db)))

+ 誤差は$Da$の分散の`len(Da)`倍と$Db$の分散の`len(Db)`倍の和となっている。

In [ ]:
print("  gosa22(a,b,D)={}\nDaの分散*len(Da)={}\nDbの分散*len(Db)={}".format(gosa22(a,b,D),bunsan(Da)*len(Da),bunsan(Db)*len(Db)))

# 4. 発展実習4  分割法で$L_2$問題2を解く

+ 方針.  
    + 実数$t$より小さいものを$D1(t)$ $t$以上を$D2(t)$とグループ分けし$a$を$D1(t)$の平均、$b$を$D2(t)$の平均とすると
       `gosa22(a,b,D)=bunsan1(D1(t))+bunsan1(D2(t))`
       
    + そこで、`gosaA2(t)=bunsan1(D1(t))+bunsan1(D2(t))`の最小値を求める.
       

+ ただし、`bunsan1`は分散にデータ数を掛けたもの.

In [ ]:
def bunsan1(D):
    return bunsan(D)*len(D)

## 4.1 誤差関数の定義

In [ ]:
def gosaA2(t,D):
    return bunsan1([d for d in D if d<t])+bunsan1([d for d in D if d>=t])


## 4.2 誤差関数のグラフ

In [ ]:
gosaA2D=lambda t:gosaA2(t,D)
showgraph([gosaA2D],["各グループの分散の和"],[0,10])

これより$3<t<6$のときに最小となることがわかる. そのとき$D1(t)=[1,2,3]$,$D2(t)=[6,7,8,9]$で $a=2,b=7.5$

In [ ]:
plot1d2(D,4)

# 5 発展実習5 $L_1$誤差についての２点近似

+ 参考実習4 と同様に行う.
+ 平均を中央値、分散を$L_1$分散にする. ただし、$L_1$分散は、中央値とデータの差の絶対値の和である。


In [ ]:
D=np.array([1,2,3,6,7,8,9])

In [ ]:
mode(D),l1bunsan(D)

## 5.1 方針

+ 実数$t$より小さいものを$D1(t)$とし、$t$以上を$D2(t)$とグループ分けし$a$を$D1(t)$の中央値、$b$を$D2(t)$の中央値とすると
       `gosa21(a,b,D)=l1bunsan(D1(t))+l1bunsan(D2(t))`
+ そこで、$gosaA1(t)=l1bunsan(D1(t))+l1bunsan(D2(t))$の最小値を求める.
       

## 5.2 誤差関数の定義

In [ ]:
def gosaA1(t,D):
    return l1bunsan([d for d in D if d<t])+l1bunsan([d for d in D if d>=t])

## 5.3 誤差関数のグラフ

In [ ]:
gosaA1D=lambda t:gosaA1(t,D)
showgraph([gosaA1D],["分割された各グループのL1分散の和"],[0,10])


## 5.4 分割の図示

+ これより、区間[3,6]にある点、たとえば、4 をとり、$4$未満と$4$以上に分離すれば良いことがわかる.この場合は$L_2$誤差の場合と同じである。

In [ ]:
plot1d2(D,4)

## 5.5 比較

In [ ]:
showgraph([gosaA2D,gosaA1D],["グループの分散和","グループのL1分散和"],[0,15])

### 5.5.1 一要素が孤立している場合(1)

+ $D$の最後の要素$9$を$16$とした場合はどうか?

In [ ]:
D[-1]=16
D

+ 誤差関数のグラフ

In [ ]:
gosaA2D=lambda u:gosaA2(u,D)
gosaA1D=lambda u:gosaA1(u,D)
showgraph([gosaA2D,gosaA1D],["L2誤差","L1誤差"],[0,20])

+ 青だけ表示すると$L_2$誤差では、[1,2,3,6,7,8]と[16]に分離するときが最小

#### 赤のグループが$16$だけになる.

In [ ]:
plot1d2(D,10)

+ $L_1$誤差では、前と同様に [1,2,3] と [6,7,8,16] に分離するときが最小となる

#### $L_1$誤差: 変化なし

In [ ]:
plot1d2(D,5)

### 5.5.2 一要素が孤立している場合(2)

+ $D$の最後の要素$9$を$20$とした場合はどうか?

In [ ]:
D[-1]=20
D

In [ ]:
gosaA2D=lambda u:gosaA2(u,D)
gosaA1D=lambda u:gosaA1(u,D)
showgraph([gosaA2D,gosaA1D],["L2誤差","L1誤差"],[0,22])

+ この場合は$L_1$誤差の場合も赤グループは"20"のみとなる.

In [ ]:
plot1d2(D,15)